In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import difflib
# from fuzzywuzzy import process

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

######################
from difflib import SequenceMatcher
from heapq import nlargest as _nlargest
def get_close_matches_indexes(word, possibilities, n=3, cutoff=0.6):
    """Use SequenceMatcher to return a list of the indexes of the best 
    "good enough" matches. word is a sequence for which close matches 
    are desired (typically a string).
    possibilities is a list of sequences against which to match word
    (typically a list of strings).
    Optional arg n (default 3) is the maximum number of close matches to
    return.  n must be > 0.
    Optional arg cutoff (default 0.6) is a float in [0, 1].  Possibilities
    that don't score at least that similar to word are ignored.
    """
    if not n >  0:
        raise ValueError("n must be > 0: %r" % (n,))
    if not 0.0 <= cutoff <= 1.0:
        raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
    result = []
    s = SequenceMatcher()
    s.set_seq2(word)
    for idx, x in enumerate(possibilities):
        s.set_seq1(x)
        if s.real_quick_ratio() >= cutoff and \
           s.quick_ratio() >= cutoff and \
           s.ratio() >= cutoff:
            result.append((s.ratio(), idx))
    # Move the best scorers to head of list
    result = _nlargest(n, result)
    # Strip scores for the best n matches
    return [x for score, x in result]
######################

nutritionDF = pd.read_csv('/kaggle/input/nutritional-values-for-common-foods-and-products/nutrition.csv', usecols= [0,1])
nutritionDF.columns = ['nutrition_id', 'prod_name']
productsDF = pd.read_csv('/kaggle/input/instacart-market-basket-analysis/products.csv',skiprows=range(1,35000),nrows=15000,header=0)

# matchedDF=productsDF.product_name.str.lower().str.replace(',', ' ').map(lambda x: difflib.get_close_matches(x, nutritionDF.prod_name.str.lower().str.replace(',', ' '),n=1, cutoff=0.4))
# matchedDF=productsDF.product_name.str.lower().replace(',', ' ').map(lambda x: process.extractOne(x, nutritionDF.prod_name.str.lower().replace(',', ' ')))
matchedDF=productsDF.product_name.str.lower().str.replace(',', ' ').map(lambda x: get_close_matches_indexes(x, nutritionDF.prod_name.str.lower().str.replace(',', ' '),n=1, cutoff=0.4))

matchedDF=matchedDF.rename('nutrition_id')
matchedDF=matchedDF.map(lambda x: x[0] if len(x)>0 else -1)
productsWithNutrientsDF=productsDF.join(matchedDF)

print(productsWithNutrientsDF[["product_name","nutrition_id"]].head())
productsWithNutrientsDF.to_csv('product_nutrient_15k_2.csv')


In [ ]:
import pandas as pd
nutritionDF = pd.read_csv('/kaggle/input/nutritional-values-for-common-foods-and-products/nutrition.csv', usecols= [0,1])
nutritionDF.columns = ['nutrition_id', 'prod_name']
prodNutrientDF=pd.read_csv('/kaggle/working/product_nutrient_15k_1.csv')
joinedDF=pd.merge(prodNutrientDF, nutritionDF,  how="left", on=["nutrition_id", "nutrition_id"])
joinedDF.to_csv('product_nutrient_joined_15k_2.csv')
joinedDF

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))